In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
from transformers import pipeline
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline, BitsAndBytesConfig , CodeGenTokenizer 
# from langchain.llms import HuggingFacePipeline 
# from langchain import PromptTemplate, LLMChain
from transformers import AutoTokenizer , AutoModelForCausalLM
import torch 
# torch.set_default_device("cuda")


In [2]:
model_id = 'mistralai/Mistral-7B-Instruct-v0.2'
# model_id = 'microsoft/phi-2'

In [3]:
# model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype="auto", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, trust_remote_code=True, device_map='cuda', load_in_4bit=True)
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True, device_map='cuda')

In [4]:

from transformers.generation import StoppingCriteria, StoppingCriteriaList

class StopWordCriteria(StoppingCriteria):
    def __init__(self, stop_word):
        self.stop_word = stop_word

    def __call__(self, input_ids, scores, **kwargs):
        text = tokenizer.decode(input_ids[0])
        return self.stop_word in text



In [9]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [15]:
prompt = '''INST: Translate the following 5 questions into German without answering them. After translating, say nothing else.
1. EN: What is the number of words in War and Peace by Leo Tolstoy?
   DE: Wie viele Wörter hat Krieg und Frieden von Leo Tolstoi?
2. EN: What is the number of episodes for the different regions in the PokÃ©mon anime?
   DE: Wie viele Episoden gibt es für die verschiedenen Regionen im PokÃ©mon-Anime?
3. EN: What is the number of hot dogs eaten by World record holder Joey Chestnut in 15 minutes?
   DE: Wie viele Hot Dogs hat der Weltrekordhalter Joey Chestnut in 15 Minuten gegessen?
4. EN: What is the number of people who have walked on the Moon?
   DE: Wie viele Menschen sind auf dem Mond gelaufen?
5. EN: What is the year that Julius Nepos becomes western Roman Emperor, deposing Glycerius?
   DE:'''

text = '''Translate the following question into German. Do not answer the question and do not say anything other than the translation.
```
What is the year that Julius Nepos becomes western Roman Emperor, deposing Glycerius?
```'''


messages = [
    # {
    #     "role": "system",
    #     "content": "You are a friendly chatbot who always responds in the style of a pirate",
    # },
    {"role": "user", "content": text},
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

inputs = tokenizer([prompt], return_tensors="pt")
inputs.to('cuda')
streamer = TextStreamer(tokenizer, skip_prompt=False, device='cuda')

with torch.no_grad():
# Despite returning the usual output, the streamer will also print the generated text to stdout.
    _ = model.generate(**inputs, streamer=streamer, max_new_tokens=50, stopping_criteria=[StopWordCriteria('<|question_end|>')], do_sample=True, temperature=0.9, top_p=0.9, top_k=0, repetition_penalty=1.1, )

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s><s> [INST] Translate the following question into German. Do not answer the question and do not say anything other than the translation.
```
What is the year that Julius Nepos becomes western Roman Emperor, deposing Glycerius?
``` [/INST] "Jahreszahl, in der Julius Nepos zum weströmischen Kaiser wird und Glycerius absetzt?"</s>
